In [2]:
import os
import json
import pandas as pd
from PIL import Image
import numpy as np

In [3]:
# Load meta_traindata file
meta_path = 'PKLot/train/_annotations.coco.json'
with open(meta_path) as file:
    meta_train = json.load(file)


In [4]:
train = pd.DataFrame(meta_train['images'])
df_meta = pd.DataFrame(meta_train['annotations'])
train = pd.DataFrame.merge(train,df_meta, how='left', left_on='id', right_on='image_id')
train = train.dropna()
train['id_y'] = train['id_y'].values.astype(int)
train['category_id'] = train['category_id'].values.astype(int)
train['area'] = train['area'].values.astype(int)
train['iscrowd'] = train['iscrowd'].values.astype(int)
train.head()

,id_x,license,file_name,height,width,date_captured,id_y,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,0,0.0,2,"[304, 468, 83.5, 114]",9519,[],0
1,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,1,0.0,2,"[271, 390, 76.5, 89]",6808,[],0
2,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,2,0.0,2,"[245, 326, 67.5, 80]",5400,[],0
3,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,3,0.0,2,"[223, 270, 63.5, 73]",4635,[],0
4,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,4,0.0,2,"[204, 224, 58, 57]",3306,[],0


In [5]:
train_img_folder = 'PKLot/grey_train'
image = {}
for img in train['file_name'].unique():
    img_path = os.path.join(train_img_folder, img)
    pixels = np.array(Image.open(img_path).convert('L'))
    pixels = pixels.reshape(640, 640)
    image[img] = pixels


In [21]:
bboxes = train[['file_name', 'bbox']].copy()
bboxes = bboxes.groupby('file_name', as_index=True).agg({'bbox': list})
bboxes

,bbox
file_name,
2012-09-11_15_16_58_jpg.rf.61d961a86c9a16694403dfcb72cd450c.jpg,"[[139, 165, 23, 40], [155, 165, 22.5, 42.5], [..."
2012-09-11_15_27_08_jpg.rf.0f38b15658ce17d10ce40a992adae5ba.jpg,"[[139, 165, 23, 40], [155, 165, 22.5, 42.5], [..."
2012-09-11_15_29_29_jpg.rf.3819d0bdff07fb09e87e5cc7d51dc662.jpg,"[[139, 165, 23, 40], [155, 165, 22.5, 42.5], [..."
2012-09-11_15_31_50_jpg.rf.b33fc3bf053074e8dab4fa16108b950c.jpg,"[[139, 165, 23, 40], [155, 165, 22.5, 42.5], [..."
2012-09-11_15_38_53_jpg.rf.bcdabdb175b85ebb981248ddc666e1d7.jpg,"[[139, 165, 23, 40], [155, 165, 22.5, 42.5], [..."
...,...
2013-04-16_10_30_05_jpg.rf.da1a752b6a0e016416d54b258fd2b74e.jpg,"[[304, 468, 83.5, 114], [271, 390, 76.5, 89], ..."
2013-04-16_10_35_05_jpg.rf.7c844923f1f3acf7ab0dee3793e47228.jpg,"[[304, 468, 83.5, 114], [271, 390, 76.5, 89], ..."
2013-04-16_10_40_05_jpg.rf.76d0f9b229a91463eafb1cd59027a281.jpg,"[[304, 468, 83.5, 114], [271, 390, 76.5, 89], ..."
